# Business Process Mining Project 
### by Ana Drmic and Leonardo Marazzi

## Description of the project

What does the notebook do

The parts that are present in the notebook

## Imports

In [1]:
import pandas as pd
import statistics
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from function import Utils
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pandas.api.types import is_numeric_dtype
import numpy as np
import h2o
from h2o.automl import H2OAutoML
from IPython.display import display
warnings.filterwarnings('ignore')

## Data loading

In [2]:
df = pd.read_csv("hmw/Sampled_BPIC17_offer_log.csv")

## Preprocessing

In the first section we wanted to map the most important featur that are mandatory to the logs:

-caseid

-activity

-resource

-ts (time stamp)

-y (target of the Machine learning study)

We then change the name of the columns to be able to use them automaticly

In [3]:
input = {
    "Case ID" : "caseid",
    "Activity" : "activity" ,
    "Resource" : "resource" ,
    "Complete Timestamp": "ts",
    "(case) Accepted" : "y"  #Needs to be 1 for each Case
}

# Change columns name
df.rename(columns=input,inplace = True)

# Timestamp value in df
df["ts"] = pd.to_datetime(df["ts"])

# This is our first function that calculate for each row the assosiated delta time
df["dt"] = Utils.dt(df) 

df.drop("ts",axis=1,inplace = True)

## Missing value analysis

We plot the persantage of missing value for each column

In [4]:
Utils.persantage_nan(df)

index                            0.000000
caseid                           0.000000
activity                         0.000000
resource                         0.000000
Variant                          0.000000
Variant index                    0.000000
y                                0.000000
(case) ApplicationID             0.000000
(case) CreditScore               0.000000
(case) FirstWithdrawalAmount     0.000000
(case) MonthlyCost               0.000000
(case) NumberOfTerms             0.000000
(case) OfferedAmount             0.000000
(case) Selected                  0.000000
Action                           0.000000
EventID                          0.000000
EventOrigin                      0.000000
OfferID                         22.312464
lifecycle:transition             0.000000
dt                               0.000000
dtype: float64

### Null value Prediction

We builded a function that given the column with null values it will train a random forest and replace the column with the predicted values and add a new column to the DataFrame to show which value are predicted and wich are not.


In [5]:
df,model_null_OfferID = Utils.predict_null_value(colum="OfferID",df=df)

In [6]:
df = Utils.prod_nan_with_treshold(df)
df.head()

,index,caseid,activity,resource,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,...,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,EventOrigin,OfferID,lifecycle:transition,dt,OfferID_was_null
0,0,Offer_247135719,O_Create Offer,User_17,Variant 8,8,True,Application_196483749,0,10000.0,...,57,10000.0,False,Created,Offer_247135719,Offer,Offer_247135719,complete,0 days 00:00:03.042000,True
1,1,Offer_247135719,O_Created,User_17,Variant 8,8,True,Application_196483749,0,10000.0,...,57,10000.0,False,statechange,OfferState_124849367,Offer,Offer_247135719,complete,0 days 00:02:12.568000,False
2,2,Offer_247135719,O_Sent (online only),User_17,Variant 8,8,True,Application_196483749,0,10000.0,...,57,10000.0,False,statechange,OfferState_440662877,Offer,Offer_247135719,complete,0 days 00:02:04.704000,False
3,3,Offer_247135719,O_Cancelled,User_17,Variant 8,8,True,Application_196483749,0,10000.0,...,57,10000.0,False,statechange,OfferState_591416028,Offer,Offer_247135719,complete,0 days 00:02:04.704000,False
4,4,Offer_941964966,O_Create Offer,User_17,Variant 1,1,True,Application_196483749,0,4100.0,...,57,10000.0,False,Created,Offer_941964966,Offer,Offer_941964966,complete,0 days 00:00:01.551000,True


## Encoding

### Group by CaseId

Afther we have preprocessed the data we can group by caseid and start working on the encoding

In [7]:
df_grouped = df.groupby(["caseid"]).agg(list).reset_index()

When we group by caseid some columns will have a list of list with only one value per row.

This function efficiently reduce these feature to one single value

In [8]:
df_grouped = Utils.reduce_list_columns(df_grouped)

The following function display the columns that need to be encoded.


In [9]:
Utils.display_columns_to_aggragate(df_grouped)

,activity,resource,Action,EventID,OfferID,dt,OfferID_was_null
0,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_20, User_20, User_20, User_117, User_115]","[Created, statechange, statechange, statechang...","[Offer_1000681710, OfferState_452902905, Offer...","[Offer_520979602, Offer_1000681710, Offer_1000...","[0 days 00:00:01.663000, 0 days 00:00:19.05600...","[True, False, False, False, False]"
1,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_2, User_2, User_2, User_113, User_30]","[Created, statechange, statechange, statechang...","[Offer_1001553250, OfferState_1604351174, Offe...","[Offer_533969936, Offer_1001553250, Offer_1001...","[0 days 00:00:01.188000, 0 days 00:00:15.47200...","[True, False, False, False, False]"
2,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_85, User_85, User_85, User_117, User_118]","[Created, statechange, statechange, statechang...","[Offer_1002136393, OfferState_1608457034, Offe...","[Offer_520979602, Offer_1002136393, Offer_1002...","[0 days 00:00:01.293000, 0 days 00:00:19.55600...","[True, False, False, False, False]"
3,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_49, User_49, User_49, User_113, User_102]","[Created, statechange, statechange, statechang...","[Offer_1002236598, OfferState_1780384890, Offe...","[Offer_1043318048, Offer_1002236598, Offer_100...","[0 days 00:00:01.269000, 0 days 00:00:11.34800...","[True, False, False, False, False]"
4,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_15, User_15, User_15, User_1]","[Created, statechange, statechange, statechange]","[Offer_1002530118, OfferState_1971804832, Offe...","[Offer_1726748288, Offer_1002530118, Offer_100...","[0 days 00:00:01.216000, 0 days 00:00:20.01800...","[True, False, False, False]"
...,...,...,...,...,...,...,...
4995,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_73, User_73, User_73, User_43]","[Created, statechange, statechange, statechange]","[Offer_993689039, OfferState_1192570219, Offer...","[Offer_862248796, Offer_993689039, Offer_99368...","[0 days 00:00:02.062000, 0 days 00:00:15.96300...","[True, False, False, False]"
4996,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_28, User_28, User_28, User_1]","[Created, statechange, statechange, statechange]","[Offer_993800442, OfferState_1754231388, Offer...","[Offer_1043318048, Offer_993800442, Offer_9938...","[0 days 00:00:01.530000, 0 days 00:00:20.28100...","[True, False, False, False]"
4997,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_19, User_19, User_19, User_116, User_113]","[Created, statechange, statechange, statechang...","[Offer_99473283, OfferState_1945701229, OfferS...","[Offer_833160233, Offer_99473283, Offer_994732...","[0 days 00:00:01.283000, 0 days 00:04:24.72900...","[True, False, False, False, False]"
4998,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_25, User_25, User_25, User_119, User_102]","[Created, statechange, statechange, statechang...","[Offer_995784215, OfferState_1350464436, Offer...","[Offer_520979602, Offer_995784215, Offer_99578...","[0 days 00:00:02.593000, 0 days 00:00:15.82700...","[True, False, False, False, False]"


Some colomns have no importance in the process so we have to drop them.

Keep in mind that if you leave a column with a lot of unique values it will make the encoding very slow, some times not feasible and may also compromise the ML model.


In [10]:
df_grouped = df_grouped.drop(['EventID','OfferID'],axis=1)
df_grouped = df_grouped.drop(['OfferID_was_null'],axis=1)

In [11]:
df_grouped.head(2)

,caseid,activity,resource,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventOrigin,lifecycle:transition,dt
0,Offer_1000681710,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_20, User_20, User_20, User_117, User_115]",Variant 2,2,True,Application_2131314372,956,14500.0,200.0,96,16000.0,True,"[Created, statechange, statechange, statechang...",Offer,complete,"[0 days 00:00:01.663000, 0 days 00:00:19.05600..."
1,Offer_1001553250,"[O_Create Offer, O_Created, O_Sent (mail and o...","[User_2, User_2, User_2, User_113, User_30]",Variant 2,2,False,Application_1607028451,0,2500.0,175.0,127,17500.0,True,"[Created, statechange, statechange, statechang...",Offer,complete,"[0 days 00:00:01.188000, 0 days 00:00:15.47200..."


## Aggregation encoding

In [12]:
df_grouped_agg = Utils.aggregation_encoding(df_grouped,df)

In [16]:
df_grouped_agg.head()

,caseid,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,...,User_86,User_142,User_76,User_141,User_32,Created,statechange,avg_dt,max_dt,min_dt
0,Offer_1000681710,Variant 2,2,True,Application_2131314372,956,14500.0,200.00,96,16000.0,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,1 days 11:21:59.362200,7 days 04:11:16.866000,0 days 00:00:01.663000
1,Offer_1001553250,Variant 2,2,False,Application_1607028451,0,2500.0,175.00,127,17500.0,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,2 days 22:47:28.444400,6 days 13:57:17.486000,0 days 00:00:01.188000
2,Offer_1002136393,Variant 2,2,True,Application_638926349,969,6500.0,343.25,20,6500.0,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,5 days 22:58:29.630800,18 days 00:58:56.495000,0 days 00:00:01.293000
3,Offer_1002236598,Variant 3,3,False,Application_818693455,0,2500.0,210.00,127,21000.0,...,0.0,0.0,0.0,0.0,0.0,0.20,0.80,1 days 20:42:58.900800,3 days 03:26:35.948000,0 days 00:00:01.269000
4,Offer_1002530118,Variant 1,1,False,Application_1126987739,0,19750.0,210.12,120,20000.0,...,0.0,0.0,0.0,0.0,0.0,0.25,0.75,15 days 06:47:37.585000,30 days 13:35:04.553000,0 days 00:00:01.216000


## Index encoding

In [17]:
df_grouped_agg = Utils.index_encoding(df_grouped,df,columns_to_encode = ["activity","resource","Action"])

In [21]:
df_grouped_agg.drop(["activity","resource","Action"],inplace=True,axis=1)

## Model Building

To train the model we decided to implement a Auto Machine learning lybrary h20o: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/welcome.html

We chose this library beacuse it is built around java that will parallelise the training as much as possible, making the process much faster.

In [18]:
import h2o
from h2o.automl import H2OAutoML
from h2o.automl import H2OAutoML

Select the dataframe you want to use

In [ ]:
df_grouped = df_grouped_agg.copy()

We first initialize the istance of h2o

In [19]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,14 mins 43 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_l_marazzi_rjo86d
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.711 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [ ]:
#Slit dataset into train and test
X_train, X_test = train_test_split(df_grouped, test_size=test_size)

#cast df into h2o DF
df_h_train = h2o.H2OFrame(X_train)
df_h_test = h2o.H2OFrame(X_test)

#select the columns for training and testing
x = df_h_train.columns
y = "y"
x.remove(y)


### Classification

In [ ]:

#Cast the y in factor for classification this is to be changed if we want to do regression
df_h_train[y] = df_h_train[y].asfactor()
df_h_train[y] = df_h_train[y].asfactor()


#Bluild and train models
aml = H2OAutoML(max_models=15, seed=1,max_runtime_secs= 120)
aml.train(x=x, y=y, training_frame=df_h_train)


Now we have all the models inside the aml object.
This object is the core of the autoML library and we can do almost everything.

#### Leaderboard

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows) 

Select the best model

In [ ]:
model = aml.leader

#### Training curve

In [ ]:
learning_curve_plot = model.learning_curve_plot()

#### Check performance

In [ ]:
perf = model.model_performance(df_h_test)
print(perf.accuracy()[0][1])
perf.confusion_matrix()

#### Model Explainability

Feature importance of a specific model

In [ ]:
h2o.get_model("XGBoost_3_AutoML_2_20240117_162729").varimp_plot()

Show feature importance between all the models

In [ ]:
va_plot = aml.varimp_heatmap()

va_plot = h2o.varimp_heatmap(aml.leaderboard.sort("auc").head(10))

# or even extended leaderboard can be used
va_plot = h2o.varimp_heatmap(h2o.automl.get_leaderboard(aml, extra_columns="training_time_ms").sort("training_time_ms").head(10))

## Model Retraining 

We can then retrain the model taking out the less important features

In [ ]:
df = pd.read_csv("/kaggle/input/data-sets/Sampled_BPIC17_offer_log.csv")

input = {
    "Case ID" : "caseid",
    "Activity" : "activity" ,
    "Resource" : "resource" ,
    "Complete Timestamp": "ts",
    "(case) Accepted" : "y"  #Needs to be 1 for each Case
}

# Change columns name
df.rename(columns=input,inplace = True)

# Timestamp value in df
df["ts"] = pd.to_datetime(df["ts"])

df["dt"] = Utils.dt(df) # Thisneed to be improved
df.drop("ts",axis=1,inplace = True)

##DROP THE USLESS COLUMNS
df.drop("resource",axis=1,inplace = True)

df = Utils.predict_null_value(colum="OfferID",df=df)

df = Utils.prod_nan_with_treshold(df)

df_grouped = df.groupby(["caseid"]).agg(list).reset_index()
df_grouped = Utils.reduce_list_columns(df_grouped)
df_grouped = df_grouped.drop(['EventID','OfferID'],axis=1)
df_grouped = df_grouped.drop(['OfferID_was_null'],axis=1)
df_grouped_agg = Utils.aggregation_encoding(df_grouped,df)
df_grouped_agg.drop("dt",axis=1,inplace = True)

In [ ]:
df_grouped_agg["max_dt"] = df_grouped_agg["max_dt"].map(lambda x : x.seconds)
df_grouped_agg["min_dt"] = df_grouped_agg["min_dt"].map(lambda x : x.seconds)
df_grouped_agg["avg_dt"] = df_grouped_agg["avg_dt"].map(lambda x : x.seconds)

In [ ]:
df_grouped_agg.drop("caseid",axis=1,inplace = True)
df_grouped_agg.drop("Variant",axis=1,inplace = True)
df_grouped_agg.drop("(case) ApplicationID",axis=1,inplace = True)

In [ ]:
h2o.init()

X_train, X_test = train_test_split(df_grouped_agg, test_size=0.3)

df_h_train = h2o.H2OFrame(X_train)
df_h_test = h2o.H2OFrame(X_test)


x = df_h_train.columns
y = "y"
x.remove(y)


df_h_train[y] = df_h_train[y].asfactor()
df_h_train[y] = df_h_train[y].asfactor()

aml = H2OAutoML(nfolds = 10, max_models=15, seed=1,max_runtime_secs= 1800,balance_classes=True)
aml.train(x=x, y=y, training_frame=df_h_train)


In [ ]:
model_2 = aml.leader
perf = model_2.model_performance(df_h_test)
print(perf.accuracy()[0][1])
perf.confusion_matrix()

In [ ]:
aml.leaderboard.head(5)

In [ ]:
h2o.get_model("GBM_3_AutoML_3_20240117_163430").varimp_plot()

In [ ]:
va_plot = aml.varimp_heatmap()

# or if some subset of the models is needed a slice of leaderboard can be used, e.g., using MAE as the sorting metric
va_plot = h2o.varimp_heatmap(aml.leaderboard.sort("auc").head(10))

# or even extended leaderboard can be used
va_plot = h2o.varimp_heatmap(h2o.automl.get_leaderboard(aml, extra_columns="training_time_ms").sort("training_time_ms").head(10))

## Evaluation of results